In [1]:
import nltk

from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
#sen = nltk.sent_tokenize(paragraph)
#words = nltk.word_tokenize(paragraph)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
#read the data
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/IMDB Dataset.csv")
# dataset = pd.read_csv('heart1.csv')
dataset

Mounted at /content/drive


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [64]:
dataset['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [56]:
data = dataset.iloc[:5000,:]

In [7]:
data.shape
#data.head()

(5000, 2)

In [48]:
#text preprocessing
#lowering case, remove stop words,punctuations and apply stemming or lemmatizing (technqiues to find root word)
#stemming - stems or cuts last few char. and might not represtn a meaningful word
#lemmatize - it takes context of word and convets to a meaningful word.helpful in chatbots(time taking process)
#from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
corpus = []
ps= PorterStemmer()
removelist=""
for i in range(len(data['review'])):

  review1 =re.sub('','',data['review'][i])          #remove HTML tags
  review2= re.sub('https://.*','',review1)   #remove URLs
  review3 = re.sub('[^a-zA-Z]', ' ',review2)
  review = review3.lower()
  review = review.split()
  reivew = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

corpus[4]


'petter mattei s love in the time of money is a visually stunning film to watch mr mattei offers us a vivid portrait about human relations this is a movie that seems to be telling us what money power and success do to people in the different situations we encounter br br this being a variation on the arthur schnitzler s play about the same theme the director transfers the action to the present time new york where all these different characters meet and connect each one is connected in one way or another to the next person but no one seems to know the previous point of contact stylishly the film has a sophisticated luxurious look we are taken to see how these people live and the world they live in their own habitat br br the only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits a big city is not exactly the best place in which human relations find sincere fulfillment as one discerns is the case with most of the people we encoun

In [ ]:
#words to numbers or vectors
#bag of words -take corpus of words and build histograms and label each word based on its frequency (word count)
#tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range = (1,2))

vec = tf.fit_transform(corpus).toarray()

In [57]:
#
data['sentiment'] = data['sentiment'].map({'positive' :1 ,'negative':0})


<ipython-input-57-d492689735e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'] = data['sentiment'].map({'positive' :1 ,'negative':0})


In [50]:
#tokenizer
from keras.preprocessing.text import Tokenizer

tok = Tokenizer()
tok.fit_on_texts(corpus)
token = tok.texts_to_sequences(corpus)

In [18]:
#c


In [51]:
#padding
from keras.preprocessing.sequence import pad_sequences

padded = pad_sequences(token,padding = 'post',maxlen =939 )

In [52]:
#busilding LSTM MODEL
from keras.layers import LSTM,Dense,Bidirectional,Embedding
from keras.models import Sequential
model = Sequential()
model.add(Embedding(vocab_size , 100,input_length = 939))
model.add(Bidirectional(LSTM(64)))
#lstm_model.add(Dropout(0.3))
model.add(Dense(1, activation = "sigmoid"))

In [53]:
vocab_size = len(tok.word_index)+1

In [54]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 939, 100)          3849700   
                                                                 
 bidirectional_2 (Bidirecti  (None, 128)               84480     
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3934309 (15.01 MB)
Trainable params: 3934309 (15.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
y = pd.DataFrame(data['sentiment'])
y.head()

,sentiment
0,1
1,1
2,1
3,0
4,1


In [59]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(padded,y,test_size =0.3)

In [60]:
model.fit(x_train,y_train,epochs = 5, validation_split=0.3, batch_size=16)

Epoch 1/5
154/154 [==============================] - 198s 1s/step - loss: 0.6619 - accuracy: 0.5861 - val_loss: 0.5683 - val_accuracy: 0.7543
Epoch 2/5
154/154 [==============================] - 191s 1s/step - loss: 0.3412 - accuracy: 0.8690 - val_loss: 0.4999 - val_accuracy: 0.7629
Epoch 3/5
154/154 [==============================] - 190s 1s/step - loss: 0.1223 - accuracy: 0.9612 - val_loss: 0.6332 - val_accuracy: 0.7314
Epoch 4/5
154/154 [==============================] - 188s 1s/step - loss: 0.0549 - accuracy: 0.9849 - val_loss: 0.9094 - val_accuracy: 0.7981
Epoch 5/5
154/154 [==============================] - 191s 1s/step - loss: 0.0273 - accuracy: 0.9931 - val_loss: 0.8072 - val_accuracy: 0.7552


In [68]:
sentence = ["The movie was very good and heart whelming",
            "I have never seen a worst movie like this",
            "the movie plot is awesome but it had good acting"]
# convert to a sequence
sequences = tok.texts_to_sequences(sentence)
# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=939)
# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
prediction


1/1 [==============================] - 0s 201ms/step


array([[0.9940172 ],
       [0.19153209],
       [0.9772269 ]], dtype=float32)

In [38]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
